In [ ]:
from datetime import datetime, timedelta
from datetime import datetime

# Reporting date range. Either parse the passed date parameter or use the value coded in the "else" case
start = datetime(2025, 1, 1, 0, 0, 0)
end = start + timedelta(days=11)

# Exclusion date ranges - this should be a list of tuples in which each tuple has two
# members, a start date and time and end date and time in that order
exclusions = []

# Whether to export the data to a spreadsheet
export_spreadsheet = True

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
# Sampling & baseline estimation
BASELINE_WINDOW = "120T"            # rolling window size for baseline (e.g. 120 minutes)
BASELINE_FUNC = "median"            # "median" or "mean"

# Excursion definition
EXCURSION_DELTA_MMOL = 1.5          # excursion = level - baseline >= this value
MIN_EXCURSION_DURATION_MIN = 20     # minimum continuous duration to count as excursion
MAX_GAP_MIN = 10                    # allow short gaps within an excursion (e.g. 10 minutes)

# For plotting
FIGSIZE = (10, 5)

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

## Data Load

In [ ]:
# Log in to the service, get the person ID and retrieve the data
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_blood_glucose_measurements(url, token, person_id, start, end)

# Remove any excluded ranges
df = remove_records_for_date_ranges(df, exclusions)

# Set the index
df = df.set_index("date").sort_index()

# Preview the data
df.head()

## Rolling Baseline

In [ ]:
# Use a centered rolling window to estimate the "local baseline"
# We use median because it's robust to transient peaks
if BASELINE_FUNC == "median":
    baseline = df["level"].rolling(BASELINE_WINDOW, center=True).median()
elif BASELINE_FUNC == "mean":
    baseline = df["level"].rolling(BASELINE_WINDOW, center=True).mean()
else:
    raise ValueError("BASELINE_FUNC must be 'median' or 'mean'")

df["baseline"] = baseline

# For edges where rolling is NaN, you can choose to:
# - back/forward fill, or
# - leave as NaN (those times won't be evaluated for excursions)
df["baseline"] = df["baseline"].interpolate("time")

df.head()

## Excursion Identification

In [ ]:
import numpy as np

# Calculate delta from baseline and excursion mask
df["delta"] = df["level"] - df["baseline"]
df["is_excursion_point"] = df["delta"] >= EXCURSION_DELTA_MMOL

# We’ll treat excursion points as belonging to the same excursion if:
# - They are consecutively True in the mask, or
# - Gaps between them are <= MAX_GAP_MIN

# Convert index to a column for easier time-delta handling
tmp = df.reset_index().rename(columns={"index": "date"})

# Identify boundaries of excursion "runs" allowing small gaps
is_exc = tmp["is_excursion_point"]

# Assign a group ID that increments when a gap exceeds MAX_GAP_MIN
group_id = []
current_group = -1
last_exc_time = None

for idx, (t, flag) in enumerate(zip(tmp["date"], is_exc)):
    if flag:
        if last_exc_time is None:
            # first excursion point
            current_group += 1
        else:
            gap_min = (t - last_exc_time).total_seconds() / 60.0
            if gap_min > MAX_GAP_MIN:
                # new excursion
                current_group += 1
        last_exc_time = t
        group_id.append(current_group)
    else:
        group_id.append(np.nan)

tmp["excursion_group"] = group_id


## Excursion Filtering

In [ ]:
# Filter out non-excursion rows and summarise by excursion_group

exc_points = tmp.dropna(subset=["excursion_group"]).copy()
exc_points["excursion_group"] = exc_points["excursion_group"].astype(int)

if exc_points.empty:
    print("No excursions detected with current thresholds")
else:
    # Compute duration per group
    group_stats = (
        exc_points
        .groupby("excursion_group")
        .agg(
            start_time=("date", "min"),
            end_time=("date", "max"),
            peak_level=("level", "max"),
            peak_delta=("delta", "max"),
            mean_delta=("delta", "mean"),
            # simple trapezoidal approximation for area under delta-time curve
            # assumes uniform sampling at RESAMPLE_RULE
        )
        .reset_index()
    )

    # duration in minutes
    group_stats["duration_min"] = (
        group_stats["end_time"] - group_stats["start_time"]
    ).dt.total_seconds() / 60.0

    # Filter by minimum duration
    group_stats = group_stats[group_stats["duration_min"] >= MIN_EXCURSION_DURATION_MIN]

    # Compute area under the curve (AUC) for delta, in mmol/L * min
    # Approximation: mean_delta * duration
    group_stats["auc_delta"] = group_stats["mean_delta"] * group_stats["duration_min"]
    group_stats = group_stats.sort_values("start_time").reset_index(drop=True)

    display(group_stats)

In [ ]:
# 08 - Attach excursion group labels back to main resampled dataframe (for plotting)

df_with_groups = tmp.set_index("date")
df_with_groups = df_with_groups[["level", "baseline", "delta", "is_excursion_point", "excursion_group"]]

df_with_groups.head()

## Plot Example Excursion

In [ ]:
import matplotlib.pyplot as plt

if not exc_points.empty and not group_stats.empty:
    first_group = group_stats.iloc[0]["excursion_group"]

    mask = df_with_groups["excursion_group"] == first_group
    excursion_df = df_with_groups[mask].copy()

    # Add some padding before and after the excursion for context
    start_time = excursion_df.index.min() - pd.Timedelta(minutes=30)
    end_time = excursion_df.index.max() + pd.Timedelta(minutes=60)

    context_df = df_with_groups.loc[start_time:end_time]

    plt.figure(figsize=FIGSIZE)
    plt.plot(context_df.index, context_df["level"], label="Glucose level")
    plt.plot(context_df.index, context_df["baseline"], linestyle="--", label="Baseline")

    # Highlight excursion region
    plt.fill_between(
        context_df.index,
        context_df["level"],
        context_df["baseline"],
        where=context_df["excursion_group"] == first_group,
        alpha=0.3,
        label="Excursion"
    )

    plt.axhline(0, linewidth=0.5)
    plt.title(f"Example Excursion (Group {first_group})")
    plt.xlabel("Time")
    plt.ylabel("Glucose (mmol/L)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("No excursions to plot with current thresholds")

## Chart Excursions Aligned at Start Time

- Typical rise speed
- Peak timing
- Recovery slope
- Outliers

In [ ]:
import matplotlib.pyplot as plt

if not exc_points.empty and not group_stats.empty:
    plt.figure(figsize=(10, 6))

    for _, row in group_stats.iterrows():
        gid = row["excursion_group"]
        start = row["start_time"]
        end = row["end_time"]

        exc = df_with_groups.loc[start:end]

        # Time axis in minutes since excursion start
        t_min = (exc.index - start).total_seconds() / 60.0

        plt.plot(
            t_min,
            exc["delta"],
            linewidth=2,
            alpha=0.35
        )

    plt.axhline(EXCURSION_DELTA_MMOL, linestyle="--", linewidth=2, label="Excursion threshold")
    plt.axhline(0, color="black", linewidth=0.5)

    plt.xlabel("Minutes since excursion start")
    plt.ylabel("Δ Glucose above baseline (mmol/L)")
    plt.title("All glucose excursions aligned at start time")
    plt.grid(True)
    plt.tight_layout()

    # Export to PNG or PDF, if required
    export_chart("glucose", "glucose_excursions_aligned", None, chart_export_format)

    plt.show()
else:
    print("No excursions to plot with current thresholds")

## Chart All Excursions on Absolute Scale

- How high do the excursions go

In [ ]:
import matplotlib.pyplot as plt

if not exc_points.empty and not group_stats.empty:
    plt.figure(figsize=(10, 6))

    for _, row in group_stats.iterrows():
        gid = row["excursion_group"]
        start = row["start_time"]
        end = row["end_time"]

        exc = df_with_groups.loc[start:end]
        t_min = (exc.index - start).total_seconds() / 60.0

        plt.plot(
            t_min,
            exc["level"],
            linewidth=2,
            alpha=0.35
        )

    plt.axhline(3.9, linestyle="--", linewidth=2, label="7.8 mmol/L reference")
    plt.axhline(10.0, linestyle="--", linewidth=2, label="10 mmol/L reference")

    plt.xlabel("Minutes since excursion start")
    plt.ylabel("Glucose (mmol/L)")
    plt.title("All glucose excursions (absolute levels)")
    plt.grid(True)
    plt.tight_layout()

    # Export to PNG or PDF, if required
    export_chart("glucose", "glucose_excursions_absolute", None, chart_export_format)

    plt.show()
else:
    print("No excursions to plot with current thresholds")

## Chart Median and Percentile Band

- Typical excursion envelope

In [ ]:
import pandas as pg
import matplotlib.pyplot as plt

if not exc_points.empty and not group_stats.empty:
    # Build aligned excursion matrix
    aligned = []

    for _, row in group_stats.iterrows():
        start = row["start_time"]
        end = row["end_time"]

        exc = df_with_groups.loc[start:end].copy()
        exc["t_min"] = (exc.index - start).total_seconds() / 60.0
        exc = exc.set_index("t_min")

        aligned.append(exc["delta"])

    # Combine into single dataframe
    aligned_df = pd.concat(aligned, axis=1)

    # Resample to common time grid (e.g. every 5 minutes)
    aligned_df = aligned_df.groupby(aligned_df.index).mean()

    median = aligned_df.median(axis=1)
    p25 = aligned_df.quantile(0.25, axis=1)
    p75 = aligned_df.quantile(0.75, axis=1)

    plt.figure(figsize=(10, 6))
    plt.plot(median.index, median, label="Median excursion")
    plt.fill_between(median.index, p25, p75, alpha=0.3, label="25–75% band")

    plt.axhline(EXCURSION_DELTA_MMOL, linestyle="--", linewidth=1, label="Threshold")
    plt.axhline(0, color="black", linewidth=0.5)

    plt.xlabel("Minutes since excursion start")
    plt.ylabel("Δ Glucose above baseline (mmol/L)")
    plt.title("Typical glucose excursion envelope")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Export to PNG or PDF, if required
    export_chart("glucose", "glucose_excursions_median", None, chart_export_format)

    plt.show()
else:
    print("No excursions to plot with current thresholds")